In [1]:
import sys
sys.path.append('..')

from helper_fun import *
import helper_fun
from Split_functions_classify.splits_two import *
from Split_functions_classify.splits_three import *
from Split_functions_classify.splits_ten import *
from joblib import load
from Split_functions_regression.split_furthest_cluster import *
SEED = 42
seed_everything(SEED)

compounds = pd.read_csv('../COVID_MOONSHOT/compounds_filtered.csv')
two_split(compounds)
three_split(compounds)
ten_split(compounds);
compound_df = compounds.copy()
compound_df["maccs"] = compound_df["SMILES"].apply(smiles_to_fp,)

/home/alex/miniforge3/envs/my-rdkit-env-covid/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-07 15:23:59.405666: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 15:23:59.443620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
RF_eval = dict()

In [3]:
fingerprint_to_model = compound_df.maccs.tolist()

In [4]:
#change variables to _2 at end
label_to_model_2 = compound_df.bin_2.tolist()

# Split data randomly in train and test set
# note that we use test/train_x for the respective fingerprint splits
# and test/train_y for the respective label splits
(
    static_train_x_2,
    static_test_x_2,
    static_train_y_2,
    static_test_y_2,
) = train_test_split(fingerprint_to_model, label_to_model_2, test_size=0.2, random_state=SEED)
splits_2 = [static_train_x_2, static_test_x_2, static_train_y_2, static_test_y_2]


#change variables to _3 at end
label_to_model_3 = compound_df.bin_3.tolist()

# Split data randomly in train and test set
# note that we use test/train_x for the respective fingerprint splits
# and test/train_y for the respective label splits
(
    static_train_x_3,
    static_test_x_3,
    static_train_y_3,
    static_test_y_3,
) = train_test_split(fingerprint_to_model, label_to_model_3, test_size=0.2, random_state=SEED)
splits_3 = [static_train_x_3, static_test_x_3, static_train_y_3, static_test_y_3]

X_train_2_furth, X_test_2_furth, Y_train_2_furth, Y_test_2_furth = furthest_cluster_split(compounds, smiles_column="SMILES", CID_column="CID", pIC50_column="bin_2")
X_train_2_furth = X_train_2_furth.to_list()
Y_train_2_furth = Y_train_2_furth.to_list()
X_test_2_furth = X_test_2_furth.to_list()
Y_test_2_furth = Y_test_2_furth.to_list()

X_train_3_furth, X_test_3_furth, Y_train_3_furth, Y_test_3_furth = furthest_cluster_split(compounds, smiles_column="SMILES", CID_column="CID", pIC50_column="bin_3")
X_train_3_furth = X_train_3_furth.to_list()
Y_train_3_furth = Y_train_3_furth.to_list()
X_test_3_furth = X_test_3_furth.to_list()
Y_test_3_furth = Y_test_3_furth.to_list()

In [5]:
# Set model parameter for random forest
model_RF_2 = RandomForestClassifier(criterion="entropy")

In [6]:
Optuna_model_RF_2 = load('Optuna_model_RF_2.pkl')

In [7]:
Optuna_model_RF_2_furth = load('Optuna_model_RF_2.pkl')

In [8]:
# Set model parameter for random forest
model_RF_3 = RandomForestClassifier(criterion="entropy")

In [9]:
Optuna_model_RF_3 = load('Optuna_model_RF_3.pkl')

In [10]:
Optuna_model_RF_3_furth = load('Optuna_model_RF_3.pkl')

In [11]:
model_RF_2.fit(static_train_x_2, static_train_y_2)
Optuna_model_RF_2.fit(static_train_x_2, static_train_y_2)
Optuna_model_RF_2_furth.fit(X_train_2_furth, Y_train_2_furth)

RandomForestClassifier(max_depth=21, n_estimators=14, random_state=42)

In [12]:
classification_report_dict_2 = classification_report(static_test_y_2, model_RF_2.predict(static_test_x_2), output_dict=True)

In [13]:
RF_2_auc = roc_auc_score(static_test_y_2, model_RF_2.predict_proba(static_test_x_2)[:, 1])
RF_2_micro_f1 = classification_report_dict_2["accuracy"]
RF_2_precision_macro = classification_report_dict_2["macro avg"]["precision"]
RF_2_recall_macro = classification_report_dict_2["macro avg"]["recall"]
RF_2_mcc = matthews_corrcoef(static_test_y_2, model_RF_2.predict(static_test_x_2))
RF_2_balanced_mcc = balanced_MCC_macro_avg(static_test_y_2, model_RF_2.predict(static_test_x_2))

In [14]:
classification_report_dict_2_optuna = classification_report(static_test_y_2, Optuna_model_RF_2.predict(static_test_x_2), output_dict=True)

In [15]:
RF_2_auc_optuna = roc_auc_score(static_test_y_2, Optuna_model_RF_2.predict_proba(static_test_x_2)[:, 1])
RF_2_micro_f1_optuna = classification_report_dict_2_optuna["accuracy"]
RF_2_precision_macro_optuna = classification_report_dict_2_optuna["macro avg"]["precision"]
RF_2_recall_macro_optuna = classification_report_dict_2_optuna["macro avg"]["recall"]
RF_2_mcc_optuna = matthews_corrcoef(static_test_y_2, Optuna_model_RF_2.predict(static_test_x_2))
RF_2_balanced_mcc_optuna = balanced_MCC_macro_avg(static_test_y_2, Optuna_model_RF_2.predict(static_test_x_2))

In [16]:
classification_report_dict_2_optuna_furth = classification_report(Y_test_2_furth, Optuna_model_RF_2_furth.predict(X_test_2_furth), output_dict=True)

In [17]:
RF_2_auc_optuna_furth = roc_auc_score(Y_test_2_furth, Optuna_model_RF_2_furth.predict_proba(X_test_2_furth)[:, 1])
RF_2_micro_f1_optuna_furth = classification_report_dict_2_optuna_furth["accuracy"]
RF_2_precision_macro_optuna_furth = classification_report_dict_2_optuna_furth["macro avg"]["precision"]
RF_2_recall_macro_optuna_furth = classification_report_dict_2_optuna_furth["macro avg"]["recall"]
RF_2_mcc_optuna_furth = matthews_corrcoef(Y_test_2_furth, Optuna_model_RF_2_furth.predict(X_test_2_furth))
RF_2_balanced_mcc_optuna_furth = balanced_MCC_macro_avg(Y_test_2_furth, Optuna_model_RF_2_furth.predict(X_test_2_furth))

In [18]:
model_RF_3.fit(static_train_x_3, static_train_y_3)
Optuna_model_RF_3.fit(static_train_x_3, static_train_y_3)
Optuna_model_RF_3_furth.fit(X_train_3_furth, Y_train_3_furth)

RandomForestClassifier(max_depth=15, max_features='log2', n_estimators=30,
                       random_state=42)

In [19]:
classification_report_dict_3 = classification_report(static_test_y_3, model_RF_3.predict(static_test_x_3), output_dict=True)

In [20]:
RF_3_auc = roc_auc_score(static_test_y_3, model_RF_3.predict_proba(static_test_x_3),average="micro", multi_class="ovr")
RF_3_micro_f1 = classification_report_dict_3["accuracy"]
RF_3_precision_macro = classification_report_dict_3["macro avg"]["precision"]
RF_3_recall_macro = classification_report_dict_3["macro avg"]["recall"]
RF_3_mcc = matthews_corrcoef(static_test_y_3, model_RF_3.predict(static_test_x_3))
RF_3_balanced_mcc = balanced_MCC_macro_avg(static_test_y_3, model_RF_3.predict(static_test_x_3))

In [21]:
classification_report_dict_3_optuna = classification_report(static_test_y_3, Optuna_model_RF_3.predict(static_test_x_3), output_dict=True)

In [22]:
RF_3_auc_optuna = roc_auc_score(static_test_y_3, Optuna_model_RF_3.predict_proba(static_test_x_3),multi_class="ovr",average="micro")
RF_3_micro_f1_optuna = classification_report_dict_3_optuna["accuracy"]
RF_3_precision_macro_optuna = classification_report_dict_3_optuna["macro avg"]["precision"]
RF_3_recall_macro_optuna = classification_report_dict_3_optuna["macro avg"]["recall"]
RF_3_mcc_optuna = matthews_corrcoef(static_test_y_3, Optuna_model_RF_3.predict(static_test_x_3))
RF_3_balanced_mcc_optuna = balanced_MCC_macro_avg(static_test_y_3, Optuna_model_RF_3.predict(static_test_x_3))

In [23]:
classification_report_dict_3_optuna_furth = classification_report(Y_test_3_furth, Optuna_model_RF_3_furth.predict(X_test_3_furth), output_dict=True)

In [24]:
RF_3_auc_optuna_furth = roc_auc_score(Y_test_3_furth, Optuna_model_RF_3_furth.predict_proba(X_test_3_furth),multi_class="ovr",average="micro")
RF_3_micro_f1_optuna_furth = classification_report_dict_3_optuna_furth["accuracy"]
RF_3_precision_macro_optuna_furth = classification_report_dict_3_optuna_furth["macro avg"]["precision"]
RF_3_recall_macro_optuna_furth = classification_report_dict_3_optuna_furth["macro avg"]["recall"]
RF_3_mcc_optuna_furth = matthews_corrcoef(Y_test_3_furth, Optuna_model_RF_3_furth.predict(X_test_3_furth))
RF_3_balanced_mcc_optuna_furth = balanced_MCC_macro_avg(Y_test_3_furth, Optuna_model_RF_3_furth.predict(X_test_3_furth))

In [25]:
RF_eval = {'Model': ['RF binary', 'RF binary Optuna', 'RF binary Optuna furthest', 'RF ternary', 'RF ternary Optuna', 'RF ternary Optuna furthest'],
           'ROC AUC': [RF_2_auc, RF_2_auc_optuna, RF_2_auc_optuna_furth, RF_3_auc, RF_3_auc_optuna, RF_3_auc_optuna_furth],
            'F1': [RF_2_micro_f1, RF_2_micro_f1_optuna, RF_2_micro_f1_optuna_furth, RF_3_micro_f1, RF_3_micro_f1_optuna, RF_3_micro_f1_optuna_furth],
            'Precision': [RF_2_precision_macro, RF_2_precision_macro_optuna, RF_2_precision_macro_optuna_furth, RF_3_precision_macro, RF_3_precision_macro_optuna, RF_3_precision_macro_optuna_furth],
            'Recall': [RF_2_recall_macro, RF_2_recall_macro_optuna, RF_2_recall_macro_optuna_furth, RF_3_recall_macro, RF_3_recall_macro_optuna, RF_3_recall_macro_optuna_furth],
            'MCC': [RF_2_mcc, RF_2_mcc_optuna, RF_2_mcc_optuna_furth, RF_3_mcc, RF_3_mcc_optuna, RF_3_mcc_optuna_furth],
            'Balanced MCC': [RF_2_balanced_mcc, RF_2_balanced_mcc_optuna, RF_2_balanced_mcc_optuna_furth, RF_3_balanced_mcc, RF_3_balanced_mcc_optuna, RF_3_balanced_mcc_optuna_furth]
           }

In [26]:
RF_eval_df=pd.DataFrame(RF_eval)
RF_eval_df.set_index('Model', inplace=True)
RF_eval_df=RF_eval_df.round(3)

In [27]:
RF_eval_df

,ROC AUC,F1,Precision,Recall,MCC,Balanced MCC
Model,,,,,,
RF binary,0.881,0.815,0.818,0.815,0.633,0.633
RF binary Optuna,0.899,0.840,0.840,0.840,0.681,0.681
RF binary Optuna furthest,0.663,0.613,0.618,0.615,0.233,0.233
RF ternary,0.921,0.790,0.790,0.802,0.689,0.705
RF ternary Optuna,0.921,0.849,0.850,0.846,0.771,0.773
RF ternary Optuna furthest,0.725,0.516,0.498,0.425,0.160,0.172
